In [1]:
import subprocess
import os
import signal
import time

In [5]:
def get_dirs(root, test_folder, test):
    pipeline = root + '/' + test_folder + '/' + test + '/model/faster_rcnn_resnet101_kenya_tracking.config'
    train_dir = root + '/' + test_folder + '/' + test + '/train'
    eval_dir = root + '/' + test_folder + '/' + test + '/eval'
    return {'config': pipeline, 'train': train_dir, 'eval': eval_dir}

def run_training(train_py, eval_py, pipeline_config, train_dir, eval_dir, name=None):  
    print('running ', name)
    %env CUDA_VISIBLE_DEVICES=0
    train_process = subprocess.Popen(['python', train_py,
                            '--logtostderr', 
                            '--pipeline_config_path=' + pipeline_config,
                            '--train_dir=' + train_dir],
                            preexec_fn=os.setsid)

    time.sleep(60)

    %env CUDA_VISIBLE_DEVICES=1
    eval_process = subprocess.Popen(['python', eval_py,
                            '--logtostderr', 
                            '--pipeline_config_path=' + pipeline_config,
                            '--checkpoint_dir=' + train_dir,
                            '--eval_dir=' + eval_dir],
                            preexec_fn=os.setsid)

    train_process.wait()

    print('sleeping')

    time.sleep(180)

    print('shutting down eval')

    os.killpg(os.getpgid(eval_process.pid), signal.SIGTERM)
    
    time.sleep(30)

In [6]:
#train 
train_py = '/home/golden/tf-models/research/object_detection/train.py' 

#eval
eval_py = '/home/golden/tf-models/research/object_detection/eval.py'  


In [7]:
root = '/home/golden/Projects_desktop/kenya-tracking/models'
test = 'IOU_threshold_tests'
point_eight = get_dirs(root, test, 'iou_thresh_0_8')
point_nine = get_dirs(root, test, 'iou_thresh_0_9')
point_nine_five = get_dirs(root, test, 'iou_thresh_0_95')

In [9]:
run_training(train_py, eval_py, point_eight['config'], point_eight['train'], point_eight['eval'], name='.8 IOU')
run_training(train_py, eval_py, point_nine['config'], point_nine['train'], point_nine['eval'], name='.9 IOU')
run_training(train_py, eval_py, point_nine_five['config'], point_nine_five['train'], point_nine_five['eval'], name='.95 IOU')



running  .8 IOU
env: CUDA_VISIBLE_DEVICES=0
env: CUDA_VISIBLE_DEVICES=1
sleeping
shutting down eval
running  .9 IOU
env: CUDA_VISIBLE_DEVICES=0
env: CUDA_VISIBLE_DEVICES=1
sleeping
shutting down eval
running  .95 IOU
env: CUDA_VISIBLE_DEVICES=0
env: CUDA_VISIBLE_DEVICES=1
sleeping
shutting down eval


In [7]:
print(full_aug_pipeline_config)

/home/golden/Projects_desktop/kenya-tracking/models/augmentation_tests/full_augmentation/model/faster_rcnn_resnet101_kenya_tracking.config


In [19]:
point_eight['config']

'/home/golden/Projects_desktop/kenya-tracking/models/IOU_threshold_tests/iou_thresh_0_8/model/faster_rcnn_resnet101_kenya_tracking.config'